In [1]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim

In [2]:
df = pd.read_excel('../References/Jadual-2-3-sungai-tercemar-2017.xlsx')
df.head()

,state,river-basin,river,no-of-stations,2016-IKA/WQI,2016-category,2016-class,2017-IKA/WQI,2017-category,2017-class
0,P.Pinang,Sg. Jawi,Sg. Jawi,1,47,T/P,IV,44,T/P,IV
1,P.Pinang,Sg. Juru,Sg. Juru,2,58,T/P,III,53,T/P,III
2,P.Pinang,Sg. Juru,Sg. Rambai,1,55,T/P,III,49,T/P,IV
3,P.Pinang/Kedah,Sg. Pinang,Sg. Jelutong,1,44,T/P,IV,49,T/P,IV
4,P.Pinang/Kedah,Sg. Pinang,Sg. Pinang,1,56,T/P,III,57,T/P,III


In [3]:
geolocator = Nominatim(user_agent="river_geocoder")

In [4]:
# insert space in camel case
def insert_spaces_in_camel_case(input_string):
    output_string = input_string[0]  # Initialize the output string with the first character

    for i in range(1, len(input_string)):
        if input_string[i].isupper() and not input_string[i - 1].isspace():
            output_string += ' '  # Add a space before the uppercase letter
        output_string += input_string[i]

    return output_string

df['river_name'] = df['river'].apply(insert_spaces_in_camel_case)

In [5]:
df['Coordinates'] = df['river_name'].apply(lambda x: geolocator.geocode(x) if x is not None else None)
mask = df['Coordinates'].isnull()
df['Coordinates'][mask] = df['river-basin'][mask].apply(lambda x: geolocator.geocode(x) if x is not None else None)


C:\Users\elvin\AppData\Local\Temp\ipykernel_26304\1850350193.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Coordinates'][mask] = df['river-basin'][mask].apply(lambda x: geolocator.geocode(x) if x is not None else None)


In [7]:
# Extract latitude and longitude from the geocoded results
df['Latitude'] = df['Coordinates'].apply(lambda x: x.latitude if x is not None else None)
df['Longitude'] = df['Coordinates'].apply(lambda x: x.longitude if x is not None else None)

a = geolocator.geocode('Pasir Gudang')
df.loc[32,'Latitude'] = a.latitude
df.loc[32,'Longitude'] = a.longitude
# Drop rows with missing coordinates
# data = data.dropna(subset=['Latitude', 'Longitude'])
df.to_excel('../References/Jadual-2-3-sungai-tercemar-2017-geocoded.xlsx')
